In [103]:

%pip install python-dotenv
%pip install openai

You should consider upgrading via the '/Users/anastasiagordeeva/projects/hack-hmt-criminal-justice/.venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/anastasiagordeeva/projects/hack-hmt-criminal-justice/.venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [104]:
import os
import sys
from dotenv import load_dotenv
load_dotenv()

True

In [67]:
import openai



account_key = os.environ['PERSONAL_KEY']  # Your Storage Account Key


# Set OpenAI API Key
openai.api_key = account_key

def return_response(message):
    client = openai.OpenAI(api_key=account_key)
    response = client.chat.completions.create(
        model="gpt-4o", 
        messages=[
            {"role": "system", "content": "You are a professional data analyst."},
            {"role": "user", "content": message}
        ],
        temperature=0.0,  # Controls randomness (0 = deterministic, 1 = very creative)
        max_tokens=5000  # Limit response length
    )
    return response

def return_response_o1(message):
    client = openai.OpenAI(api_key=account_key)
    response = client.chat.completions.create(
        model="o1-preview", 
        messages=[
            {"role": "user", "content": message}
        ],
        max_completion_tokens=32768,  # Limit response length,
    )
    return response




In [7]:
import pandas as pd
csv_registry_df = pd.read_csv("../data/csv_registry_V2.csv")
csv_registry = csv_registry_df.to_dict(orient='records')


In [8]:
csv_registry = [el for el in csv_registry if el.get('clean_csv_path') is not None]

In [9]:
[el for el in csv_registry if el['clean_csv_path'] == '../data/clean_csv/Population_31Dec2019.xlsx_1.4.csv']

[{'file_name': 'prison/population/Population_31Dec2019.xlsx',
  'file_id': '0x8DD46BAC05B0B5A',
  'file_size': 132195,
  'file_path': '../data/Population_31Dec2019.xlsx',
  'csv_title': '1.4',
  'active_row_number': 2,
  'active_column_number': 0,
  'dataset_description': 'This dataset provides information on the prison population in England and Wales, disaggregated by ethnic group and sex, over multiple time periods in 2018 and 2019. It includes counts of individuals in prison, as well as percentage changes over time.',
  'dataset_column_names': "['Ethnic Group and Sex', '2018-12-31', '2019-03-31', '2019-06-30', '2019-09-30', '2019-12-31', 'Percentage Change (2018-2019)', 'Notes']",
  'time_coverage_start': '2018-Q4',
  'time_coverage_end': '2019-Q4',
  'geo_coverage': "['England', 'Wales']",
  'dataset_topic': 'The dataset focuses on the prison population by ethnic group and sex in England and Wales.',
  'dataset_aggregation': 'The dataset aggregates prison population counts by ethni

In [21]:
import time

In [280]:
batch_size = 10

def process_batch(batch, research_question):
    # Your processing logic here
    filename_description_dict = str({el['clean_csv_path']: el['full_description']
                                 for el in batch})
    
    research_query = f"""{research_question}

Based on descriptions of the datasets provided, select and output the file_names of the datasets that are HIGHLY RELEVANT to the research task at hand as a list of string file names.

If none seems relevant, output empty list like [] with NO ADDITIONAL COMMENTS.

Do NOT return any JSON markdown in the answer, return a simple string with square brackets and comma separated file names like so: ['file_name1', 'file_name2', 'file_name3']

Descriptions of the datasets (as file_name - to - description map): {filename_description_dict}

Output:

"""
    response = return_response(research_query)
    return(response)


# Iterate over csv_registry in batches of 5
all_related_datasets = []
research_question = """I want to build a timeseries model for tracing how prosecutions translate into prison sentences. 
I want to analyse data by quarter in England and Wales between 2020 and 2024"""

for i in range(0, len(csv_registry), batch_size):
    batch = csv_registry[i:i + batch_size]
    related_datasets = process_batch(batch, research_question)
    time.sleep(2)
    print(related_datasets.choices[0].message.content)
    related_datasets = eval(related_datasets.choices[0].message.content)
    all_related_datasets.extend(related_datasets)
    

['../data/clean_csv/cjs-overview-tables-June-2024.ods_Contents.csv', '../data/clean_csv/cjs-overview-tables-June-2024.ods_Q1_1.csv']
['../data/clean_csv/cjs-overview-tables-June-2024.ods_Q1_3.csv', '../data/clean_csv/cjs-overview-tables-June-2024.ods_Q1_4.csv']
['../data/clean_csv/cjs-overview-tables-June-2024.ods_Q5_1a.csv', '../data/clean_csv/cjs-overview-tables-June-2024.ods_Q5_1b.csv', '../data/clean_csv/cjs-overview-tables-June-2024.ods_Q5_2a.csv', '../data/clean_csv/cjs-overview-tables-June-2024.ods_Q5_2b.csv', '../data/clean_csv/cjs-overview-tables-June-2024.ods_Q5_2c.csv', '../data/clean_csv/cjs-overview-tables-June-2024.ods_Q5_3.csv']
['../data/clean_csv/cjs-overview-tables-June-2024.ods_Q5_4.csv', '../data/clean_csv/CPS-Court-Caseload-Tables-Q2-24-25.xlsx_Title.csv', '../data/clean_csv/CPS-Court-Caseload-Tables-Q2-24-25.xlsx_Index.csv', '../data/clean_csv/CPS-Court-Caseload-Tables-Q2-24-25.xlsx_1.1.csv', '../data/clean_csv/CPS-Court-Caseload-Tables-Q2-24-25.xlsx_1.2.csv', '..

In [281]:
csv_registry_df[csv_registry_df['clean_csv_path'].isin(all_related_datasets)].to_csv('related_datasets.csv', index=False)

In [282]:
def dataset_collation_prompt(dataset_list, research_question): 
    full_dataset_descriptions_lst = []
    i = 0
    for el in csv_registry:
        if el['clean_csv_path'] in dataset_list:
            el_copy = deepcopy(el)
            el_copy.pop('file_name')
            el_copy.pop('file_path')
            el_copy.pop('full_description')

            i += 1
            full_dataset_descriptions_lst.append(f"dataset {str(i)} - {str(el_copy)}")

    full_dataset_descriptions = " ".join(full_dataset_descriptions_lst)
    
    final_prompt = f""" You are a senior data analyst specializing in data cleaning and transformation. 
    Based on the research question, csv decriptions and a list of csv files, return a data modelling plan that outlines the following:
    * Out of all the datasets provided, selects the relevant datasets based on the list of csv files, and if multiple suitable similar csvs are present. Specify which ones are using by the clean_csv_path.
    * Performs data cleaning operations to ensure the datasets are in a usable format.
    * If multiple tables in CSV, split tables and process separately.
    * When aggregating by quarter, ensure that the quarter date is the LAST date in the quarter.
    * Standartizes column and row naming conventions of the datasets with short, distinct names. Checks that the number of columns matches he assigned list of standard column names to avoid errors.
    * Performs data joins via pandas merge and concat functions to unify the datasets into one dataframe.

    It should be in plain text with some pandas functions suggested (no code examples needed).

    In addition, provide a short description of each column in the resulting dataframe using the revised column names.

    The research question is: {research_question}
    
    The datasets are as follows: {full_dataset_descriptions}

    """

    response = return_response_o1(final_prompt)
    response_text = response.choices[0].message.content

    return(response_text)

    
    

In [283]:
from copy import deepcopy

In [284]:
research_question = research_question
response = dataset_collation_prompt(all_related_datasets, research_question)

In [285]:
print(response)

**Data Modeling Plan**

**1. Selection of Relevant Datasets**

Based on the research question—to build a time series model tracing how prosecutions translate into prison sentences in England and Wales between 2020 and 2024—we will use the following datasets:

- **Prosecutions Data:**
  - **Dataset 22:** `../data/clean_csv/Prosecution-Data-Tables-Q2-24-25.xlsx_1.2.csv`
    - Provides quarterly statistics on prosecutions, convictions, and non-convictions from 2023-Q3 to 2024-Q2.
  - **Dataset 25:** `../data/clean_csv/Prosecution-Data-Tables-Q3-23-24.xlsx_1.2.csv`
    - Covers quarterly data from 2022-Q4 to 2023-Q3.
  - **Dataset 24:** `../data/clean_csv/Prosecution-Data-Tables-Q3-23-24.xlsx_1.2.csv`
    - Includes data from 2022-Q3.

- **Prison Sentences Data:**
  - **Dataset 7:** `../data/clean_csv/cjs-overview-tables-June-2024.ods_Q5_2a.csv`
    - Contains the number of offenders sentenced to immediate custody by quarter from 2014-Q2 to 2024-Q2.

**Note:** We have selected these datase

In [254]:
import re

# Load datasets
# Datasets used (with clean_csv_path):
# Dataset 1: '../data/clean_csv/Prosecution-Data-Tables-Q2-24-25.xlsx_1.2.csv'
df1 = pd.read_csv('../data/clean_csv/Prosecution-Data-Tables-Q2-24-25.xlsx_1.2.csv')
df1 = df1.iloc[1:, 2:].T
df1.columns = df1.iloc[0]
df1 = df1.iloc[1:]
# Dataset 2: '../data/clean_csv/Prosecution-Data-Tables-Q2-24-25.xlsx_1.3.csv'
df2 = pd.read_csv('../data/clean_csv/Prosecution-Data-Tables-Q2-24-25.xlsx_1.3.csv')
df2_index = df2.iloc[1,:]
df2 = df2.iloc[1:,:]
df2.columns = df2_index
df2['']
# Dataset 3: '../data/clean_csv/Prosecution-Data-Tables-Q2-24-25.xlsx_1.4.csv'
df3 = pd.read_csv('../data/clean_csv/Prosecution-Data-Tables-Q2-24-25.xlsx_1.4.csv')
# Dataset 4: '../data/clean_csv/Prosecution-Data-Tables-Q2-24-25.xlsx_1.5.csv'
df4 = pd.read_csv('../data/clean_csv/Prosecution-Data-Tables-Q2-24-25.xlsx_1.5.csv')

# Perform data cleaning and standardization
def standardize_columns(df):
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(' ', '_')
        .str.replace('%', 'percent')
        .str.replace('/', '_')
        .str.replace('[^a-zA-Z0-9_]', '', regex=True)
    )
    return df

df_list = [df1, df2, df3, df4]
df_list = [standardize_columns(df) for df in df_list]

In [273]:
df2 = pd.read_csv('../data/clean_csv/Prosecution-Data-Tables-Q2-24-25.xlsx_1.3.csv')


Total completed prosecutions,Total completed prosecutions,Convictions,% Convictions,Non-convictions,% Non-convictions
Convictions,104195,85882,82.4%,18313,17.6%
% Convictions,109841,90963,82.8%,18878,17.2%
Non-convictions,108620,90479,83.3%,18141,16.7%
% Non-convictions,111670,92502,82.8%,19168,17.2%
Rolling year to date,434326,359826,82.8%,74500,17.2%


In [275]:
df2 = df2.iloc[1:,:]

In [277]:
df2.columns = df2.iloc[0]
df2 = df2.iloc[1:]

In [278]:
df2.columns.rename({'Unnamed: 1': 'quarter'}, inplace=True)
df2['quarter'] = ['']

1,Unnamed: 1,Total completed prosecutions,Convictions,% Convictions,Guilty pleas,% Guilty pleas,Convictions after trial,% Convictions after trial,Proved in absence,% Proved in absence,Non-convictions,% Non-convictions,Prosecutions dropped,% Prosecutions dropped,Acquitted/dismissed during or after trial,% Acquitted/dismissed during or after trial,Administratively finalised,% Administratively finalised,Discharged,% Discharged
2,Unnamed: 2,104195,85882,82.4%,79535,76.3%,5030,4.8%,1317,1.3%,18313,17.6%,11740,11.3%,3269,3.1%,3298,3.2%,6,0.0%
3,Unnamed: 3,109841,90963,82.8%,84758,77.2%,4791,4.4%,1414,1.3%,18878,17.2%,12267,11.2%,3408,3.1%,3199,2.9%,4,0.0%
4,Unnamed: 4,108620,90479,83.3%,84233,77.5%,4946,4.6%,1300,1.2%,18141,16.7%,11813,10.9%,3297,3.0%,3026,2.8%,5,0.0%
5,Unnamed: 5,111670,92502,82.8%,86218,77.2%,4874,4.4%,1410,1.3%,19168,17.2%,12206,10.9%,3354,3.0%,3605,3.2%,3,0.0%
6,Unnamed: 6,434326,359826,82.8%,334744,77.1%,19641,4.5%,5441,1.3%,74500,17.2%,48026,11.1%,13328,3.1%,13128,3.0%,18,0.0%


In [257]:
# Select relevant columns and ensure consistency
for df in df_list:
    df.columns = ['quarter'] + list(df.columns[1:])

# Keep only the necessary columns
df1 = df1[['quarter', 'total_completed_prosecutions', 'convictions', 'percent_convictions', 'nonconvictions', 'percent_nonconvictions',]]
df2 = df2[['quarter', 'total_completed_prosecutions', 'convictions', 'percent_convictions', 'nonconvictions', 'percent_nonconvictions', ]]
df3 = df3[['quarter', 'total_completed_prosecutions', 'convictions', 'percent_convictions', 'non_convictions', 'percent_non_convictions', ]]
df4 = df4[['quarter', 'total_completed_prosecutions', 'convictions', 'percent_convictions', 'non_convictions', 'percent_non_convictions', ]]

# Concatenate datasets
combined_df = pd.concat(df_list, ignore_index=True)

# Function to map quarter labels to the last date in the quarter
def quarter_to_date(quarter_label):
    # Pattern matches fiscal year format '23/24-Q3'
    match = re.match(r'(\d{2})/(\d{2})-Q(\d)', quarter_label)
    if match:
        start_year = int(match.group(1)) + 2000
        end_year = int(match.group(2)) + 2000
        quarter = int(match.group(3))
        # Map fiscal quarters to end months
        q_end_month = {1: '09-30', 2: '12-31', 3: '03-31', 4: '06-30'}
        # Determine the correct year for the quarter end
        if quarter == 1 or quarter == 2:
            year = end_year
        else:
            year = start_year + 1
        date_str = f"{year}-{q_end_month[quarter]}"
        return date_str
    else:
        return None

KeyError: "['total_completed_prosecutions', 'convictions', 'percent_convictions', 'non_convictions', 'percent_non_convictions'] not in index"

In [256]:
df3

,quarter,total_completed_prosecutions,convictions,percent_convictions,guilty_pleas,percent_guilty_pleas,convictions_after_trial,percent_convictions_after_trial,proved_in_absence,percent_proved_in_absence,nonconvictions,percent_nonconvictions,prosecutions_dropped,percent_prosecutions_dropped,acquitted_dismissed_during_or_after_trial,percent_acquitted_dismissed_during_or_after_trial,administratively_finalised,percent_administratively_finalised,discharged,percent_discharged
1,Unnamed: 1,Total completed prosecutions,Convictions,% Convictions,Guilty pleas,% Guilty pleas,Convictions after trial,% Convictions after trial,Proved in absence,% Proved in absence,Non-convictions,% Non-convictions,Prosecutions dropped,% Prosecutions dropped,Acquitted/dismissed during or after trial,% Acquitted/dismissed during or after trial,Administratively finalised,% Administratively finalised,Discharged,% Discharged
2,Unnamed: 2,104195,85882,82.4%,79535,76.3%,5030,4.8%,1317,1.3%,18313,17.6%,11740,11.3%,3269,3.1%,3298,3.2%,6,0.0%
3,Unnamed: 3,109841,90963,82.8%,84758,77.2%,4791,4.4%,1414,1.3%,18878,17.2%,12267,11.2%,3408,3.1%,3199,2.9%,4,0.0%
4,Unnamed: 4,108620,90479,83.3%,84233,77.5%,4946,4.6%,1300,1.2%,18141,16.7%,11813,10.9%,3297,3.0%,3026,2.8%,5,0.0%
5,Unnamed: 5,111670,92502,82.8%,86218,77.2%,4874,4.4%,1410,1.3%,19168,17.2%,12206,10.9%,3354,3.0%,3605,3.2%,3,0.0%
6,Unnamed: 6,434326,359826,82.8%,334744,77.1%,19641,4.5%,5441,1.3%,74500,17.2%,48026,11.1%,13328,3.1%,13128,3.0%,18,0.0%


In [207]:
df.dropna(how='all', inplace=True)
df.dropna(axis=1, how='all', inplace=True)

In [208]:
df

,Unnamed: 0,5,6,7,8,10,12,13,14,15
0,Unnamed: 0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,The following tables show flagged cases and th...,Month,Magistrates' & Crown Courts received,Magistrates' & Crown Courts carried forward,Magistrates' & Crown Courts finalised prosecut...,Table 2 - Prosecution caseload: Adult rape fla...,Month,Magistrates' & Crown Courts received,Magistrates' & Crown Courts carried forward,Magistrates' & Crown Courts finalised prosecut...
2,Unnamed: 2,19-04,43265,98392,38186,NaN,19-04,165,708,106
3,Unnamed: 3,19-05,45261,99578,39950,NaN,19-05,149,697,85
4,Unnamed: 4,19-06,41378,100581,37092,NaN,19-06,171,726,83
...,...,...,...,...,...,...,...,...,...,...
63,Unnamed: 63,24-05,44934,166749,37299,NaN,24-05,502,3608,188
64,Unnamed: 64,24-06,42025,169663,34752,NaN,24-06,450,3656,220
65,Unnamed: 65,24-07,46127,169781,39927,NaN,24-07,564,3689,270
66,Unnamed: 66,24-08,43502,172596,35909,NaN,24-08,519,3765,231


In [161]:
# Step 1: Select relevant datasets
# We have identified the following datasets as relevant to the research question:

# Dataset 2: Recorded crime and notifiable offence outcomes
path_crime = '../data/clean_csv/cjs-overview-tables-June-2024.ods_Q1_2.csv'

# Dataset 4: Quarterly statistics on prosecutions and convictions
path_prosecutions = '../data/clean_csv/cjs-overview-tables-June-2024.ods_Q1_4.csv'

# Dataset 5: Prosecution caseload data for Magistrates' and Crown Courts
path_caseload = '../data/clean_csv/CPS-Court-Caseload-Tables-Q2-24-25.xlsx_1.1.csv'


In [164]:
# Step 2: Load datasets
df_crime = pd.read_csv(path_crime)
df_prosecutions = pd.read_csv(path_prosecutions)
df_caseload = pd.read_csv(path_caseload)

# Step 3: Data cleaning and formatting

# Function to standardize quarter dates to the last date of the quarter
def standardize_quarter_dates(df, date_column):
    df[date_column] = pd.to_datetime(df[date_column]) + pd.tseries.offsets.QuarterEnd()
    return df

# Clean df_crime
# Remove unnecessary headers, reset index if needed
df_crime = df_crime.dropna(how='all')  # Remove empty rows
df_crime.columns = df_crime.iloc[0]  # Set first row as header
df_crime = df_crime[1:]  # Drop the header row from data
df_crime = df_crime.iloc[:,1:]  # Drop the header row from data
df_crime.columns = ['Category'] + [str(int(year)) for year in range(2015, 2025)] + ['PercentageChange']
# Filter data for years 2018-2024
df_crime = df_crime[['Category'] + [str(year) for year in range(2018, 2025)]]


In [165]:
df_crime

,Category,2018,2019,2020,2021,2022,2023,2024
1,Recorded crime [note 19][note 20],5608281.0,6018191.0,5793145.0,5765269.0,6466644.0,6719737.0,6664047.0
2,Recorded crime: Excluding fraud [note 21],4970212.0,5293548.0,5029182.0,4845996.0,5449527.0,5563646.0,5366753.0
3,Charged/summonsed [note 19][note 20],462908.0,440215.0,428713.0,367704.0,352097.0,375368.0,414079.0
4,Out of court disposals (OOCDs) [note 6],224128.0,212666.0,221503.0,207981.0,205180.0,203926.0,202745.0
5,OOCD: penalty notices for disorder (notifiable...,10010.0,8825.0,9208.0,7575.0,4937.0,4085.0,3022.0
6,"OOCD: cautions (all offence, notifiable only)",88857.0,78779.0,70766.0,63617.0,61224.0,56633.0,54465.0
7,OOCD: cannabis / khat warnings [note 7],29121.0,19728.0,17455.0,12631.0,5517.0,3090.0,1285.0
8,OOCD: community resolutions [note 9],96140.0,105334.0,124074.0,124158.0,133502.0,140118.0,143973.0
9,"Proceedings (all offence, notifiable only) [no...",720692.0,670181.0,548228.0,627346.0,551908.0,611276.0,740525.0
10,"Convictions (all offence, notifiable only) [no...",568643.0,525777.0,414515.0,450465.0,428128.0,431914.0,496876.0


In [142]:
magistrates_df = pd.read_csv('../data/clean_csv/CPS-Court-Caseload-Tables-Q2-24-25.xlsx_1.2.csv')
magistrates_df = magistrates_df[magistrates_df['CPS Area'] != 'CPS Area']  # Remove header row
magistrates_df = magistrates_df[magistrates_df['CPS Area'] == 'National']
magistrates_long = magistrates_df.melt(id_vars='CPS Area', var_name='Date', value_name='MagistratesReceived')
magistrates_long = magistrates_long[~(magistrates_long['Date'].str.contains('Unnamed'))]
magistrates_long['Date'] = pd.to_datetime(magistrates_long['Date'], format='%y-%m')
magistrates_long['MagistratesReceived'] = pd.to_numeric(magistrates_long['MagistratesReceived'], errors='coerce')
magistrates_long = magistrates_long.set_index('Date')

magistrates_quarterly = magistrates_long.resample('Q').sum()
magistrates_quarterly.index = magistrates_quarterly.index.to_period('Q').to_timestamp(how='start')
magistrates_quarterly = magistrates_quarterly.reset_index()



/var/folders/my/g2y6kcc55995mrbr97lxgk9r0000gn/T/ipykernel_20867/3455544439.py:10: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  magistrates_quarterly = magistrates_long.resample('Q').sum()


In [143]:
magistrates_quarterly

,Date,CPS Area,MagistratesReceived
0,2019-04-01,NationalNationalNationalNationalNationalNation...,396932
1,2019-07-01,NationalNationalNationalNationalNationalNation...,387763
2,2019-10-01,NationalNationalNationalNationalNationalNation...,375760
3,2020-01-01,NationalNationalNationalNationalNationalNation...,384825
4,2020-04-01,NationalNationalNationalNationalNationalNation...,445734
5,2020-07-01,NationalNationalNationalNationalNationalNation...,572780
6,2020-10-01,NationalNationalNationalNationalNationalNation...,544586
7,2021-01-01,NationalNationalNationalNationalNationalNation...,495005
8,2021-04-01,NationalNationalNationalNationalNationalNation...,458961
9,2021-07-01,NationalNationalNationalNationalNationalNation...,424501


In [145]:
# Read Crown Courts caseload data
crown_court_df = pd.read_csv('../data/clean_csv/CPS-Court-Caseload-Tables-Q2-24-25.xlsx_1.3.csv')
crown_court_df = crown_court_df[crown_court_df['CPS Area'] != 'CPS Area']  # Remove header row
crown_court_df = crown_court_df[crown_court_df['CPS Area'] == 'National']
crown_court_long = crown_court_df.melt(id_vars='CPS Area', var_name='Date', value_name='CrownCourtReceived')
crown_court_long = crown_court_long[~(crown_court_long['Date'].str.contains('Unnamed'))]

crown_court_long['Date'] = pd.to_datetime(crown_court_long['Date'], format='%y-%m')
crown_court_long['CrownCourtReceived'] = pd.to_numeric(crown_court_long['CrownCourtReceived'], errors='coerce')
crown_court_long = crown_court_long.set_index('Date')

crown_court_quarterly = crown_court_long.resample('Q').sum()
crown_court_quarterly.index = crown_court_quarterly.index.to_period('Q').to_timestamp(how='start')
crown_court_quarterly = crown_court_quarterly.reset_index()


/var/folders/my/g2y6kcc55995mrbr97lxgk9r0000gn/T/ipykernel_20867/2603938468.py:12: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  crown_court_quarterly = crown_court_long.resample('Q').sum()


In [146]:
# Merge magistrates and crown court data
caseload_df = pd.merge(magistrates_quarterly, crown_court_quarterly, on='Date', how='outer')

# Merge with unemployment data
merged_df = pd.merge(caseload_df, unemployment_df, on='Date', how='left')

# Extract 'Year' from 'Date' for crime data merge
merged_df['Year'] = merged_df['Date'].dt.year
# Merge with total crime data
merged_df = pd.merge(merged_df, total_crime, on='Year', how='left')

In [148]:
unemployment_df

,Unnamed: 0,Date,UnemploymentRate
188,195,2018-01-01,4.2
189,196,2018-04-01,4.0
190,197,2018-07-01,4.1
191,198,2018-10-01,4.0
192,199,2019-01-01,3.8
193,200,2019-04-01,3.9
194,201,2019-07-01,3.9
195,202,2019-10-01,3.7
196,203,2020-01-01,4.1
197,204,2020-04-01,4.1


In [147]:
merged_df

,Date,CPS Area_x,MagistratesReceived,CPS Area_y,CrownCourtReceived,Unnamed: 0,UnemploymentRate,Year,TotalCrime
0,2019-04-01,NationalNationalNationalNationalNationalNation...,396932,NationalNationalNationalNationalNationalNation...,146751,200,3.9,2019,6018191.0
1,2019-07-01,NationalNationalNationalNationalNationalNation...,387763,NationalNationalNationalNationalNationalNation...,151883,201,3.9,2019,6018191.0
2,2019-10-01,NationalNationalNationalNationalNationalNation...,375760,NationalNationalNationalNationalNationalNation...,156592,202,3.7,2019,6018191.0
3,2020-01-01,NationalNationalNationalNationalNationalNation...,384825,NationalNationalNationalNationalNationalNation...,165065,203,4.1,2020,5793145.0
4,2020-04-01,NationalNationalNationalNationalNationalNation...,445734,NationalNationalNationalNationalNationalNation...,159043,204,4.1,2020,5793145.0
5,2020-07-01,NationalNationalNationalNationalNationalNation...,572780,NationalNationalNationalNationalNationalNation...,197024,205,5.0,2020,5793145.0
6,2020-10-01,NationalNationalNationalNationalNationalNation...,544586,NationalNationalNationalNationalNationalNation...,222995,206,5.3,2020,5793145.0
7,2021-01-01,NationalNationalNationalNationalNationalNation...,495005,NationalNationalNationalNationalNationalNation...,236266,207,4.9,2021,5765269.0
8,2021-04-01,NationalNationalNationalNationalNationalNation...,458961,NationalNationalNationalNationalNationalNation...,242381,208,4.7,2021,5765269.0
9,2021-07-01,NationalNationalNationalNationalNationalNation...,424501,NationalNationalNationalNationalNationalNation...,240790,209,4.4,2021,5765269.0


In [136]:
unemployment_df

,Unnamed: 0,Date,UnemploymentRate
188,195,2018-01-01,4.2
189,196,2018-04-01,4.0
190,197,2018-07-01,4.1
191,198,2018-10-01,4.0
192,199,2019-01-01,3.8
193,200,2019-04-01,3.9
194,201,2019-07-01,3.9
195,202,2019-10-01,3.7
196,203,2020-01-01,4.1
197,204,2020-04-01,4.1


In [135]:
merged_df

,Date,CPS Area_x,MagistratesReceived,CPS Area_y,CrownCourtReceived,Unnamed: 0,UnemploymentRate,Year,TotalCrime
0,2019-06-30,NationalNationalNationalNationalNationalNation...,396932,NationalNationalNationalNationalNationalNation...,146751,NaN,NaN,2019,6018191.0
1,2019-09-30,NationalNationalNationalNationalNationalNation...,387763,NationalNationalNationalNationalNationalNation...,151883,NaN,NaN,2019,6018191.0
2,2019-12-31,NationalNationalNationalNationalNationalNation...,375760,NationalNationalNationalNationalNationalNation...,156592,NaN,NaN,2019,6018191.0
3,2020-03-31,NationalNationalNationalNationalNationalNation...,384825,NationalNationalNationalNationalNationalNation...,165065,NaN,NaN,2020,5793145.0
4,2020-06-30,NationalNationalNationalNationalNationalNation...,445734,NationalNationalNationalNationalNationalNation...,159043,NaN,NaN,2020,5793145.0
5,2020-09-30,NationalNationalNationalNationalNationalNation...,572780,NationalNationalNationalNationalNationalNation...,197024,NaN,NaN,2020,5793145.0
6,2020-12-31,NationalNationalNationalNationalNationalNation...,544586,NationalNationalNationalNationalNationalNation...,222995,NaN,NaN,2020,5793145.0
7,2021-03-31,NationalNationalNationalNationalNationalNation...,495005,NationalNationalNationalNationalNationalNation...,236266,NaN,NaN,2021,5765269.0
8,2021-06-30,NationalNationalNationalNationalNationalNation...,458961,NationalNationalNationalNationalNationalNation...,242381,NaN,NaN,2021,5765269.0
9,2021-09-30,NationalNationalNationalNationalNationalNation...,424501,NationalNationalNationalNationalNationalNation...,240790,NaN,NaN,2021,5765269.0


In [137]:
# Merge magistrates and crown court data
caseload_df = pd.merge(magistrates_quarterly, crown_court_quarterly, on='Date', how='outer')

# Merge with unemployment data
merged_df = pd.merge(caseload_df, unemployment_df, on='Date', how='left')

In [140]:
unemployment_df['Date'].value_counts()

Date
2018-01-01    1
2021-07-01    1
2024-04-01    1
2024-01-01    1
2023-10-01    1
2023-07-01    1
2023-04-01    1
2023-01-01    1
2022-10-01    1
2022-07-01    1
2022-04-01    1
2022-01-01    1
2021-10-01    1
2021-04-01    1
2018-04-01    1
2021-01-01    1
2020-10-01    1
2020-07-01    1
2020-04-01    1
2020-01-01    1
2019-10-01    1
2019-07-01    1
2019-04-01    1
2019-01-01    1
2018-10-01    1
2018-07-01    1
2024-07-01    1
Name: count, dtype: int64

In [139]:
caseload_df['Date'].value_counts()

Date
2019-06-30    1
2019-09-30    1
2024-06-30    1
2024-03-31    1
2023-12-31    1
2023-09-30    1
2023-06-30    1
2023-03-31    1
2022-12-31    1
2022-09-30    1
2022-06-30    1
2022-03-31    1
2021-12-31    1
2021-09-30    1
2021-06-30    1
2021-03-31    1
2020-12-31    1
2020-09-30    1
2020-06-30    1
2020-03-31    1
2019-12-31    1
2024-09-30    1
Name: count, dtype: int64

In [151]:
merged_df

,Date,CPS Area_x,MagistratesReceived,CPS Area_y,CrownCourtReceived,Unnamed: 0,UnemploymentRate,Year,TotalCrime
0,2019-04-01,NationalNationalNationalNationalNationalNation...,396932,NationalNationalNationalNationalNationalNation...,146751,200,3.9,2019,6018191.0
1,2019-07-01,NationalNationalNationalNationalNationalNation...,387763,NationalNationalNationalNationalNationalNation...,151883,201,3.9,2019,6018191.0
2,2019-10-01,NationalNationalNationalNationalNationalNation...,375760,NationalNationalNationalNationalNationalNation...,156592,202,3.7,2019,6018191.0
3,2020-01-01,NationalNationalNationalNationalNationalNation...,384825,NationalNationalNationalNationalNationalNation...,165065,203,4.1,2020,5793145.0
4,2020-04-01,NationalNationalNationalNationalNationalNation...,445734,NationalNationalNationalNationalNationalNation...,159043,204,4.1,2020,5793145.0
5,2020-07-01,NationalNationalNationalNationalNationalNation...,572780,NationalNationalNationalNationalNationalNation...,197024,205,5.0,2020,5793145.0
6,2020-10-01,NationalNationalNationalNationalNationalNation...,544586,NationalNationalNationalNationalNationalNation...,222995,206,5.3,2020,5793145.0
7,2021-01-01,NationalNationalNationalNationalNationalNation...,495005,NationalNationalNationalNationalNationalNation...,236266,207,4.9,2021,5765269.0
8,2021-04-01,NationalNationalNationalNationalNationalNation...,458961,NationalNationalNationalNationalNationalNation...,242381,208,4.7,2021,5765269.0
9,2021-07-01,NationalNationalNationalNationalNationalNation...,424501,NationalNationalNationalNationalNationalNation...,240790,209,4.4,2021,5765269.0


In [286]:
[el for el in csv_registry if el['clean_csv_path'] == '../data/clean_csv/cjs-overview-tables-June-2024.ods_Q1_1.csv']

[{'file_name': 'CJS_overview/cjs-overview-tables-June-2024.ods',
  'file_id': '0x8DD46BBB6AF20D4',
  'file_size': 125388,
  'file_path': '../data/cjs-overview-tables-June-2024.ods',
  'csv_title': 'Q1_1',
  'active_row_number': 3,
  'active_column_number': 1,
  'dataset_description': 'The dataset provides statistics on individuals entering the Criminal Justice System in England and Wales over a 10-year period, from the 12 months ending June 2014 to the 12 months ending June 2024. It includes data on various stages of the justice process, such as out-of-court disposals, defendants proceeded against, offenders convicted, and prison receptions.',
  'dataset_column_names': "['Category', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024', 'Percentage Change']",
  'time_coverage_start': '2014-Q2',
  'time_coverage_end': '2024-Q2',
  'geo_coverage': "['England', 'Wales']",
  'dataset_topic': 'Statistics on individuals entering the Criminal Justice System in